# HW 2

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler,QuantileTransformer
from scipy.spatial.distance import cdist
from sklearn import linear_model, datasets
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import accuracy_score

In [2]:
import Lowess as Lowess
import Logistic as LowessLR

## Part 2
Implement your own version of Locally Weighted Logistic Regression and compare its performance on the Iris data set with the version presented in this article: https://calvintchi.github.io/classical_machine_learning/2020/08/16/lwlr.html.

### Kernels

In [3]:
# Defining Kernels

# Gaussian Kernel
def Gaussian(x):
  return np.where(np.abs(x)>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*x**2))
    
# Tricubic Kernel
def Tricubic(x):
  return np.where(np.abs(x)>1,0,(1-np.abs(x)**3)**3)
    
# Epanechnikov Kernel
def Epanechnikov(x):
  return np.where(np.abs(x)>1,0,3/4*(1-np.abs(x)**2))
    
# Quartic Kernel
def Quartic(x):
  return np.where(np.abs(x)>1,0,15/16*(1-np.abs(x)**2)**2)

### Importing iris data, TTS, and Scaling

In [4]:
# importing data from sklearn
iris = datasets.load_iris()

In [5]:
# defining x and y variables
X = iris.data
y = iris.target

In [6]:
# performing test-train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [7]:
# scaling data using MinMax Scaler, can also use other scalers which are currently commented out
scaler = MinMaxScaler()
#scaler = StandardScaler()
#scaler = QuantileTransformer()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Model Training and Evaluation

In [8]:
# creating a LocallyWeightedLogisticRegression model with Gaussian kernel, learing rate of  0.01, and tau=0.05
model = LowessLR.LocallyWeightedLogisticRegression(kernel=Gaussian, lr=0.01, tau=0.05) # accuracy of .97

#model = LowessLR.LocallyWeightedLogisticRegression(kernel=Tricubic, lr=0.01, tau=0.05) # accuracy of .97
#model = LowessLR.LocallyWeightedLogisticRegression(kernel=Epanechnikov, lr=0.01, tau=0.05) # accuracy of .97
#model = LowessLR.LocallyWeightedLogisticRegression(kernel=Quartic, lr=0.01, tau=0.05) # accuracy of .97

In [9]:
# fitting model to training data
model.fit(X_train, y_train)

In [10]:
# using trained model to make predictions on test data
predictions = model.predict(X_test)

In [11]:
# Calculating accuracy of predictions compared to the true labels (y_test) and printing results
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 96.67%


## Comparing results to Calvin Chi Paper

In [12]:
# version presented in Calvin Chi article

class locally_weighted_logistic_regression(object):
    
    def __init__(self, tau, reg = 0.0001, threshold = 1e-6):
        self.reg = reg
        self.threshold = threshold
        self.tau = tau
        self.w = None
        self.theta = None
        self.x = None

    def weights(self, x_train, x):
        sq_diff = (x_train - x)**2
        norm_sq = sq_diff.sum(axis = 1)
        return np.ravel(np.exp(- norm_sq / (2 * self.tau**2)))

    def logistic(self, x_train):
        return np.ravel(1 / (1 + np.exp(-x_train.dot(self.theta))))

    def train(self, x_train, y_train, x):
        self.w = self.weights(x_train, x)
        self.theta = np.zeros(x_train.shape[1])
        self.x = x
        gradient = np.ones(x_train.shape[1]) * np.inf
        while np.linalg.norm(gradient) > self.threshold:
            # compute gradient
            h = self.logistic(x_train)
            gradient = x_train.T.dot(self.w * (np.ravel(y_train) - h)) - self.reg * self.theta
            # Compute Hessian
            D = np.diag(-(self.w * h * (1 - h)))
            H = x_train.T.dot(D).dot(x_train) - self.reg * np.identity(x_train.shape[1])
            # weight update
            self.theta = self.theta - np.linalg.inv(H).dot(gradient)
    
    def predict(self,x):  # adjusted slightly to allow for input feature
        return np.array(self.logistic(x) > 0.5).astype(int)

In [13]:
# training one v rest models
model_dict = {}  # initialize dictionary to store models

# training a model for each class
for cls in np.unique(y_train):  # iterating over each unique class label in the training data
    binary_y_train = (y_train == cls).astype(int)  # creating binary target variable for current class
    model = locally_weighted_logistic_regression(tau=.05)  # initializing model
    model.train(X_train, binary_y_train, X_train)  # training  model
    model_dict[cls] = model  # storing model for given class

In [14]:
# making predictions
predictions = []  # initializing list to store predictions

for x_test in X_test: # iterating over each test sample
    class_probs = [] # initializing list to store probabilities for each class
    
    for cls, model in model_dict.items():  # iterating over the items in model_dict
        prob = model.predict(x_test.reshape(1, -1))  # calling predict method of current model to get prediction
        class_probs.append(prob[0])  # appends predicted probability for current class to class_probs list
    
    # after evaluating all classes for current test sample, determine class with highest probability
    predictions.append(np.argmax(class_probs))

In [15]:
# Calculating accuracy of predictions compared to the true labels (y_test) and printing results

accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 83.33%
